<h1>Quotes Classification</h1>

In [ ]:
pip install simpletransformers

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
try:
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import tensorflow as tf

Task was destroyed but it is pending!
task: <Task pending name='Task-1' coro=<Kernel.poll_control_queue() running at C:\Users\Om Verma\Desktop\multiclass-text-classification\venv\lib\site-packages\ipykernel\kernelbase.py:279> wait_for=<Future finished result=<Future at 0x...state=pending>> cb=[_chain_future.<locals>._call_set_state() at C:\Program Files\Python310\lib\asyncio\futures.py:392]>


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import os
import re
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:

df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/duplicates/train3.csv')

In [ ]:
df.columns

In [ ]:
df.columns = ['Quote','classification']

In [ ]:
df['classification'].value_counts()

In [ ]:
df['labels'] = df.classification.astype('category').cat.codes

In [ ]:
df['labels'].value_counts()

In [ ]:
df.drop(df.loc[df['labels']==-1].index, inplace=True)

In [ ]:
df['labels'].value_counts()

In [ ]:
rem_list = ['{', '}', '(2)', '(3)', '(4)', '(5)', '(6)', '\n'] 
def clean_lyrics(x):
    for p in rem_list:
        x = str(x).replace(p, '').strip()
    return x

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')


In [ ]:
def clean_lyrics(x):
  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(x)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  filtered_sentence = []
  for w in word_tokens:
    if w not in stop_words:
      filtered_sentence.append(w)
  x=filtered_sentence
  str1=""
  for ele in x:
    str1+=ele
    str1+=" "
  x=str1
  return x

In [ ]:
df['quotes_cleaned'] = df['Quote'].apply(clean_lyrics)
df['token_count'] = df['quotes_cleaned'].apply(lambda x: len(str(x).split()))
df_model = df[['quotes_cleaned', 'labels']]
df_model.columns = ['Quote', 'labels']

In [ ]:
df_model.head(20)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_model, test_size=0.2, random_state=89, stratify = df_model['labels'])

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
model_args = ClassificationArgs()
model_args.num_train_epochs = 5
model_args.learning_rate = 1e-5
model_args.overwrite_output_dir= True
model = ClassificationModel('roberta', 'roberta-base', num_labels=5, use_cuda=True, args=model_args)
model.train_model(train)

In [ ]:
from sklearn.metrics import classification_report
test_predictions, raw_outputs = model.predict(test['Quote'].to_list())
print(classification_report(test['labels'].to_list(),test_predictions))

<h1> evaluating classifier </h1>

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(test,
                                                            f1=f1_multiclass,
                                                            acc=accuracy_score)

In [ ]:
result